In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import shutil
import sys
import numpy as np
from scipy import sparse
import matplotlib.pyplot as plt
import seaborn as sn
import pandas as pd
import tensorflow as tf
import bottleneck as bn
import keras
import math

In [ ]:
import argparse
import pickle
import time
import numpy as np

In [ ]:
cd /content/drive/MyDrive/Colab Notebooks/recommendation_system/deep_learning/session_based_rec/TAGNN-master/

In [ ]:
train_data = pickle.load(open('../datasets/' + 'diginetica' + '/train.txt', 'rb'))

In [ ]:
train_data

In [ ]:

def build_graph(train_data):
    graph = nx.DiGraph()
    for seq in train_data:
        for i in range(len(seq) - 1):
            if graph.get_edge_data(seq[i], seq[i + 1]) is None:
                weight = 1
            else:
                weight = graph.get_edge_data(seq[i], seq[i + 1])['weight'] + 1
            graph.add_edge(seq[i], seq[i + 1], weight=weight)
    for node in graph.nodes:
        sum = 0
        for j, i in graph.in_edges(node):
            sum += graph.get_edge_data(j, i)['weight']
        if sum != 0:
            for j, i in graph.in_edges(i):
                graph.add_edge(j, i, weight=graph.get_edge_data(j, i)['weight'] / sum)
    return graph

'''
def data_masks(all_usr_pois, item_tail):
    us_lens = [len(upois) for upois in all_usr_pois]
    len_max = max(us_lens)
    us_pois = [upois + item_tail * (len_max - le) for upois, le in zip(all_usr_pois, us_lens)]
    us_msks = [[1] * le + [0] * (len_max - le) for le in us_lens]
    return us_pois, us_msks, len_max
'''

def data_masks(all_usr_pois, item_tail):

    us_lens = [len(upois) for upois in all_usr_pois]
    print("us_lens in data_masks")
    print(us_lens,'\n')
    
    len_max = max(us_lens)
    print("len_max in data_masks")
    print(len_max,'\n')
    us_pois = [upois + item_tail * (len_max - le) for upois, le in zip(all_usr_pois, us_lens)]
    #print('us_pois in data_masks')
    #print(us_pois)
    us_msks = [[1] * le + [0] * (len_max - le) for le in us_lens]
    return us_pois, us_msks, len_max




def split_validation(train_set, valid_portion):
    train_set_x, train_set_y = train_set
    n_samples = len(train_set_x)
    sidx = np.arange(n_samples, dtype='int32')
    np.random.shuffle(sidx)
    n_train = int(np.round(n_samples * (1. - valid_portion)))
    valid_set_x = [train_set_x[s] for s in sidx[n_train:]]
    valid_set_y = [train_set_y[s] for s in sidx[n_train:]]
    train_set_x = [train_set_x[s] for s in sidx[:n_train]]
    train_set_y = [train_set_y[s] for s in sidx[:n_train]]

    return (train_set_x, train_set_y), (valid_set_x, valid_set_y)


class Data():
    def __init__(self, data, shuffle=False, graph=None):
        inputs = data[0]
        inputs, mask, len_max = data_masks(inputs, [0])
        self.inputs = np.asarray(inputs)
        self.mask = np.asarray(mask)
        self.len_max = len_max
        self.targets = np.asarray(data[1])
        self.length = len(inputs)
        self.shuffle = shuffle
        self.graph = graph

    def generate_batch(self, batch_size):
        if self.shuffle:
            shuffled_arg = np.arange(self.length)
            np.random.shuffle(shuffled_arg)
            self.inputs = self.inputs[shuffled_arg]
            self.mask = self.mask[shuffled_arg]
            self.targets = self.targets[shuffled_arg]
        n_batch = int(self.length / batch_size)
        if self.length % batch_size != 0:
            n_batch += 1
        slices = np.split(np.arange(n_batch * batch_size), n_batch)
        slices[-1] = slices[-1][:(self.length - batch_size * (n_batch - 1))]
        return slices

    def get_slice(self, i):
        inputs, mask, targets = self.inputs[i], self.mask[i], self.targets[i]
        '''
        print("##### in get_slice, inputs")
        print(inputs)
        print(len(inputs),len(inputs[0]),len(inputs[1]))
        print("##### in get_slice, mask")
        print(mask)
        print("##### in get_slice, targets")
        print(targets)
        '''
        items, n_node, A, alias_inputs = [], [], [], []
        for u_input in inputs:
            n_node.append(len(np.unique(u_input))) 
        max_n_node = np.max(n_node) # 해당 slices 중 한 sequence 안에서 가질 수 있는 아이템의 종류의 최댓값
        # 0 패딩 후에 개수를 따지는 것이기 때문에 id 개수 +1 임.
        
        for u_input in inputs:
            node = np.unique(u_input)
            items.append(node.tolist() + (max_n_node - len(node)) * [0])
            u_A = np.zeros((max_n_node, max_n_node)) # 균일한 크기로 생성.
            for i in np.arange(len(u_input) - 1):
                if u_input[i + 1] == 0:
                    break
                u = np.where(node == u_input[i])[0][0]
                v = np.where(node == u_input[i + 1])[0][0]
                u_A[u][v] = 1
            u_sum_in = np.sum(u_A, 0)
            u_sum_in[np.where(u_sum_in == 0)] = 1
            u_A_in = np.divide(u_A, u_sum_in)
            u_sum_out = np.sum(u_A, 1)
            u_sum_out[np.where(u_sum_out == 0)] = 1
            u_A_out = np.divide(u_A.transpose(), u_sum_out)
            u_A = np.concatenate([u_A_in, u_A_out]).transpose()
            A.append(u_A)
            alias_inputs.append([np.where(node == i)[0][0] for i in u_input])
        return alias_inputs, A, items, mask, targets


In [ ]:
train_data, valid_data = split_validation(train_data, 0.1)
test_data = valid_data

In [ ]:
train_data1 = Data(train_data, shuffle=False)
test_data1 = Data(test_data, shuffle=False)

In [ ]:
class opt:
  def __init__(self,step,hidden_size,batch_size,nonhybrid=None):
    self.step=step
    self.hiddenSize = hidden_size
    #self.n_node = n_node
    self.batchSize = batch_size
    self.nonhybrid = nonhybrid
    self.lr = 0.001
    self.l2 = 1e-5
    self.lr_dc_step = 3
    self.lr_dc = 0.1
    self.epoch = 30
    self.patience = 10
    

In [ ]:
oopt=opt(step = 1,hidden_size=100,batch_size=100)

In [ ]:
n_node = 43098

In [ ]:
slices = train_data1.generate_batch(100)

In [ ]:
iiter = 0
# slice 한 단위가 size 100의 minibatch임.,
for i, j in zip(slices, np.arange(len(slices))):
    alias_inputs, A, items, mask, targets = train_data1.get_slice(i)
    print("iiter",iiter)
    print("alias_inputs",'\n')
    #print(alias_inputs.shape,'\n')
    print(len(alias_inputs[0]),'\n') # alias_inputs -> 모든 경우 중에서 가장 긴 sequence? max len 20으로 제한 한거 아니었나 ?
    print(alias_inputs[0]) 
    print("A[0]",'\n')
    print(len(A[0]),'\n')
    print(len(A[0][0]),'\n') # row 길이의 2배 -> in out을 따지기 때문.
    print(A[0])
    print("A[1]",'\n')
    print(len(A[1]),'\n')
    print(len(A[1][0]),'\n') # row 길이의 2배 -> in out을 따지기 때문.
    print(A[1],'\n')
    print("items[0]",'\n')
    print(len(items[0]),'\n')
    print(items[0],'\n') # items는 item 종류를 오름차순으로 나열한 vector -> 이때 첫 원소는 무조건 0인데 이는 padding이 이루어져있기 때문이다.
    # 또한 한 batch 기준으로 가장 긴 sequence 내의 종류를 많이 가진 sequence로부터 A와 items의 길이를 같은 길이로 가지고 있다.
    print("mask")
    print(mask.shape)
    print(mask)
    
    iiter +=1
    if iiter == 3 :
      break

In [ ]:
class GNN(tf.keras.layers.Layer):
    def __init__(self, hidden_size, step=1):
        super(GNN, self).__init__()
        self.step = step
        self.hidden_size = hidden_size
        self.input_size = hidden_size * 2
        self.gate_size = 3 * hidden_size
        # self.w_ih = Parameter(torch.Tensor(self.gate_size, self.input_size))
        self.stdv = 1/math.sqrt(self.hidden_size)
        self.w_ih = self.add_weight(shape=(self.input_size,self.gate_size),
                               initializer=tf.keras.initializers.RandomUniform(-self.stdv,self.stdv))
        '''
        AttributeError: module 'keras.api._v2.keras.initializers' has no attribute 'Uniform'
        add_weight doesn't have the attribute named 'kernel_initializer'
        '''
        # self.w_hh = Parameter(torch.Tensor(self.gate_size, self.hidden_size))
        self.w_hh = self.add_weight(shape=(self.hidden_size,self.gate_size),
                               initializer=tf.keras.initializers.RandomUniform(-self.stdv,self.stdv))
        # self.b_ih = Parameter(torch.Tensor(self.gate_size))
        self.b_ih = self.add_weight(shape=(self.gate_size,),
                               initializer=tf.keras.initializers.RandomUniform(-self.stdv,self.stdv))
        
        # self.b_hh = Parameter(torch.Tensor(self.gate_size))
        self.b_hh = self.add_weight(shape=(self.gate_size,),
                               initializer=tf.keras.initializers.RandomUniform(-self.stdv,self.stdv))
                
        # self.b_iah = Parameter(torch.Tensor(self.hidden_size))
        self.b_iah = self.add_weight(shape=(self.hidden_size,),
                               initializer=tf.keras.initializers.RandomUniform(-self.stdv,self.stdv))
        # self.b_oah = Parameter(torch.Tensor(self.hidden_size))
        self.b_oah = self.add_weight(shape=(self.hidden_size,),
                               initializer=tf.keras.initializers.RandomUniform(-self.stdv,self.stdv))

        # self.linear_edge_in = nn.Linear(self.hidden_size, self.hidden_size, bias=True)
        self.linear_edge_in = tf.keras.layers.Dense(self.hidden_size, 
                                                    use_bias=True,
                               kernel_initializer=tf.keras.initializers.RandomUniform(-self.stdv,self.stdv),
                               bias_initializer=tf.keras.initializers.RandomUniform(-self.stdv,self.stdv))
        '''
        TypeError: ('Unknown keyword argument:', 'bias') - > use_bias=True in Dense
        '''
        # self.linear_edge_out = nn.Linear(self.hidden_size, self.hidden_size, bias=True)
        self.linear_edge_out = tf.keras.layers.Dense(self.hidden_size,
                                    use_bias=True ,
                               kernel_initializer=tf.keras.initializers.RandomUniform(-self.stdv,self.stdv),
                               bias_initializer=tf.keras.initializers.RandomUniform(-self.stdv,self.stdv))
        # self.linear_edge_f = nn.Linear(self.hidden_size, self.hidden_size, bias=True)
        self.linear_edge_f = tf.keras.layers.Dense(self.hidden_size,
                                    use_bias=True ,
                               kernel_initializer=tf.keras.initializers.RandomUniform(-self.stdv,self.stdv),
                               bias_initializer=tf.keras.initializers.RandomUniform(-self.stdv,self.stdv))

    def Cell(self, A, hidden):
        input_in = tf.matmul(A[:, :, :A.shape[1]], self.linear_edge_in(hidden)) + self.b_iah
        input_out = tf.matmul(A[:, :, A.shape[1]: 2 * A.shape[1]],self.linear_edge_out(hidden)) + self.b_oah
        inputs = tf.concat([input_in, input_out], 2) 
        gi = tf.matmul(inputs, self.w_ih )+ self.b_ih # Dense layer와 같은 역할함.
        # gate_size가 앞서서 3* hidden_size로 정의되어 있었고 아래 chunk에서 split하는 것 같다.
        gh = tf.matmul(hidden, self.w_hh) + self.b_hh
        i_r, i_i, i_n = tf.split(gi,3,axis = 2)
        h_r, h_i, h_n = tf.split(gh,3,axis = 2)
        resetgate = tf.nn.sigmoid(i_r + h_r)
        inputgate = tf.nn.sigmoid(i_i + h_i)
        newgate = tf.nn.tanh(i_n + resetgate * h_n)
        hy = newgate + inputgate * (hidden - newgate)
        return hy

    def call(self, A, hidden):
        for i in range(self.step):
            hidden = self.Cell(A, hidden)
        return hidden

In [ ]:

class TAGNN(tf.keras.models.Model):
    def __init__(self, opt, n_node):
        super(TAGNN, self).__init__()
        self.hidden_size = opt.hiddenSize
        self.n_node = n_node
        self.batch_size = opt.batchSize
        self.nonhybrid = opt.nonhybrid
        # self.embedding = nn.Embedding(self.n_node, self.hidden_size)
        self.embedding = tf.keras.layers.Embedding(input_dim = self.n_node,output_dim=self.hidden_size)
        self.stdv = 1/math.sqrt(self.hidden_size)
        print(self.stdv)
        self.gnn = GNN(self.hidden_size, step=opt.step)

        # self.linear_one = nn.Linear(self.hidden_size, self.hidden_size, bias=True)
        self.linear_one = tf.keras.layers.Dense(self.hidden_size, use_bias=True,
                                                kernel_initializer=tf.keras.initializers.RandomUniform(-self.stdv,self.stdv),
                                                bias_initializer=tf.keras.initializers.RandomUniform(-self.stdv,self.stdv))
        # self.linear_two = nn.Linear(self.hidden_size, self.hidden_size, bias=True)
        self.linear_two = tf.keras.layers.Dense(self.hidden_size, use_bias=True,
                                                kernel_initializer=tf.keras.initializers.RandomUniform(-self.stdv,self.stdv),
                                                bias_initializer=tf.keras.initializers.RandomUniform(-self.stdv,self.stdv))
        # self.linear_three = nn.Linear(self.hidden_size, 1, bias=False)
        self.linear_three = tf.keras.layers.Dense(1 , use_bias=False,
                                                  kernel_initializer=tf.keras.initializers.RandomUniform(-self.stdv,self.stdv),
                                                  )
        # self.linear_transform = nn.Linear(self.hidden_size * 2, self.hidden_size, bias=True)
        
        self.linear_transform = tf.keras.layers.Dense(self.hidden_size, use_bias=True,
                                                      kernel_initializer=tf.keras.initializers.RandomUniform(-self.stdv,self.stdv),
                                                      bias_initializer=tf.keras.initializers.RandomUniform(-self.stdv,self.stdv))

        # self.linear_t = nn.Linear(self.hidden_size, self.hidden_size, bias=False)  #target attention
        self.linear_t = tf.keras.layers.Dense( self.hidden_size, use_bias=False,
                                              kernel_initializer=tf.keras.initializers.RandomUniform(-self.stdv,self.stdv),
                                              )

    def call(self, inputs, A, mask,alias_inputs):
        hidden = self.embedding(inputs) 
        hidden = self.gnn(A, hidden)
        get = lambda j: tf.gather(hidden[j],alias_inputs[j])
        hidden = tf.stack([get(j) for j in range(len(alias_inputs))])
        ht = tf.gather_nd(hidden ,indices = tf.stack([tf.range(mask.shape[0],dtype='int64') , tf.cast(tf.math.reduce_sum(mask,1),'int64') -1],axis=1))
        # tf.reshape을 이용해야함.
        q1 = tf.reshape(self.linear_one(ht) ,[ht.shape[0], 1, ht.shape[1]])  # batch_size x 1 x latent_size
        q2 = self.linear_two(hidden)  # batch_size x seq_length x latent_size
        alpha = self.linear_three(tf.nn.sigmoid(q1 + q2))  # (b,s,1)
        
        alpha = tf.nn.softmax(alpha, 1) # B,S,1
        
        
        a = tf.math.reduce_sum(alpha * hidden * tf.cast(tf.reshape(mask, [mask.shape[0], -1, 1]),dtype = 'float32'), 1)  # (b,d)
        if not self.nonhybrid: # 여기 적용되는지 판단할 것
        
            a = self.linear_transform(tf.concat([a, ht], 1)) 
        
        b = self.embedding.weights[0][1:] # 이 차원이 맞긴 함.

        hidden = hidden * tf.cast(tf.reshape(mask,[mask.shape[0], -1, 1]),'float32')  # batch_size x seq_length x latent_size
        

        qt = self.linear_t(hidden)  # batch_size x seq_length x latent_size

        beta = tf.nn.softmax(b @ tf.transpose(qt,[0,2,1]), -1)  # batch_size x n_nodes x seq_length
        target = beta @ hidden  # batch_size x n_nodes x latent_size
        a = tf.reshape(a,[ht.shape[0], 1, ht.shape[1]])  # b,1,d
        a = a + target  # b,n,d # 이거 concat인가 ? 

        scores = tf.math.reduce_sum(a * b, -1)  # b,n
        scores=tf.nn.softmax(scores,-1)
        return scores

In [ ]:
model=TAGNN(oopt,n_node)

loss_ftn = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam(learning_rate = oopt.lr)


#@tf.function
def train_ftn(item_input,A_input,mask_input,alias_inputs_,targets_input):
    with tf.GradientTape() as tape :
      scores = model(item_input, A_input, mask_input,alias_inputs_,training =True)
      
      ll = loss_ftn(targets_input-1,scores)

    
    
    gradients =tape.gradient(ll,model.trainable_weights)

    optimizer.apply_gradients(zip(gradients,model.trainable_weights))
    return ll

In [ ]:
for epoch in range(oopt.epoch):
    print('-------------------------------------------------------')
    print('epoch: ', epoch)

    slices = train_data1.generate_batch(model.batch_size)
    loss = 0
    for i, j in zip(slices, np.arange(len(slices))):
        alias_inputs, A, items, mask, targets = train_data1.get_slice(i)

        alias_inputs = tf.convert_to_tensor(alias_inputs)
        items = tf.convert_to_tensor(items)
        
        A = tf.convert_to_tensor(A)
        mask = tf.convert_to_tensor(mask)
        targets = tf.convert_to_tensor(targets)
        
        l = train_ftn(items,A,mask,alias_inputs,targets)
        
        loss += l 
        
        if j % int(100) == 0:
            print('[%d/%d] Loss: %.4f' % (j, len(slices), l))
    print('\tLoss:\t%.3f' % loss)
    hit, mrr = [], []
    slices = test_data1.generate_batch(model.batch_size)
    for i in slices:
        alias_inputs, A, items, mask, targets = test_data1.get_slice(i)
        
        alias_inputs = tf.convert_to_tensor(alias_inputs)
        items = tf.convert_to_tensor(items)
        
        A = tf.convert_to_tensor(A)
        mask = tf.convert_to_tensor(mask)
        targets = tf.convert_to_tensor(targets)

        scores = model(items, A, mask,alias_inputs,training =False)
        scores = tf.math.top_k(scores,k=20).indices
        scores = scores.numpy()
        
            # sub_scores를 tensor에서 numpy로 바꾸고 아래는 그대로 실행하면 됨.
        for score, target, mask in zip(scores, targets, test_data1.mask):
            hit.append(np.isin(target - 1, score)) # 현재 target은 s_{n+1}인 item이므로, 
                # score 즉, 위에서 추출한 index가 일치하면 hit이라는 list에 append 해준다.

                # score vector에서 target-1과 일치하는 경우를 찾고, 없는 경우 0을 mrr이라는 list에 기록하고,
            if len(np.where(score == target - 1)[0]) == 0:
                mrr.append(0)
            else:
                # 있는 경우에는 (1/그 위치의 index +1)을 기록한다.
                mrr.append(1 / (np.where(score == target - 1)[0][0] + 1))
    hit = np.mean(hit) * 100
    mrr = np.mean(mrr) * 100

    print(f"Recall@20 : {hit} , MRR@20 {mrr}")

```
-------------------------------------------------------
epoch:  0
[0/6476] Loss: 10.6742
[100/6476] Loss: 10.3441
[200/6476] Loss: 10.3890
[300/6476] Loss: 10.0419
[400/6476] Loss: 10.1113
[500/6476] Loss: 10.0773
[600/6476] Loss: 10.0492
[700/6476] Loss: 9.8870
[800/6476] Loss: 9.9263
[900/6476] Loss: 9.4517
[1000/6476] Loss: 9.5802
[1100/6476] Loss: 9.4094
[1200/6476] Loss: 9.2726
[1300/6476] Loss: 9.5695
[1400/6476] Loss: 8.9833
[1500/6476] Loss: 8.9375
[1600/6476] Loss: 8.9565
[1700/6476] Loss: 9.0298
[1800/6476] Loss: 8.7707
[1900/6476] Loss: 8.6235
[2000/6476] Loss: 8.5580
[2100/6476] Loss: 7.7815
[2200/6476] Loss: 8.1085
[2300/6476] Loss: 8.3937
[2400/6476] Loss: 8.5145
[2500/6476] Loss: 8.9056
[2600/6476] Loss: 7.9689
[2700/6476] Loss: 8.1868
[2800/6476] Loss: 7.5975
[2900/6476] Loss: 7.0928
[3000/6476] Loss: 7.6673
[3100/6476] Loss: 7.2652
[3200/6476] Loss: 7.1488
[3300/6476] Loss: 7.6578
[3400/6476] Loss: 6.7352
[3500/6476] Loss: 7.5735
[3600/6476] Loss: 7.1185
[3700/6476] Loss: 6.7363
[3800/6476] Loss: 7.1752
[3900/6476] Loss: 6.8210
[4000/6476] Loss: 6.8594
[4100/6476] Loss: 7.2792
[4200/6476] Loss: 7.0095
[4300/6476] Loss: 6.5061
[4400/6476] Loss: 6.4664
[4500/6476] Loss: 6.3978
[4600/6476] Loss: 6.3706
[4700/6476] Loss: 6.1495
[4800/6476] Loss: 6.1316
[4900/6476] Loss: 6.6278
[5000/6476] Loss: 6.3290
[5100/6476] Loss: 6.3069
[5200/6476] Loss: 5.8040
[5300/6476] Loss: 6.8233
[5400/6476] Loss: 5.9865
[5500/6476] Loss: 6.3105
[5600/6476] Loss: 6.5764
[5700/6476] Loss: 6.8081
[5800/6476] Loss: 6.3562
[5900/6476] Loss: 6.2396
[6000/6476] Loss: 6.0869
[6100/6476] Loss: 6.3947
[6200/6476] Loss: 6.2522
[6300/6476] Loss: 6.3709
[6400/6476] Loss: 6.3870
	Loss:	49852.270
Recall@20 : 44.17140395012996 , MRR@20 14.631633569161318
-------------------------------------------------------
epoch:  1
[0/6476] Loss: 5.8395
[100/6476] Loss: 5.4635
[200/6476] Loss: 5.6273
[300/6476] Loss: 5.3756
[400/6476] Loss: 5.9251
[500/6476] Loss: 5.6455
[600/6476] Loss: 5.7672
[700/6476] Loss: 5.6056
[800/6476] Loss: 5.6116
[900/6476] Loss: 5.5962
[1000/6476] Loss: 5.8353
[1100/6476] Loss: 5.6936
[1200/6476] Loss: 5.6624
[1300/6476] Loss: 5.9936
[1400/6476] Loss: 5.3912
[1500/6476] Loss: 5.2994
[1600/6476] Loss: 5.5733
[1700/6476] Loss: 6.1238
[1800/6476] Loss: 5.8300
[1900/6476] Loss: 5.6951
[2000/6476] Loss: 5.3237
[2100/6476] Loss: 4.8946
[2200/6476] Loss: 5.0202
[2300/6476] Loss: 5.2621
[2400/6476] Loss: 6.0669
[2500/6476] Loss: 6.4053
[2600/6476] Loss: 5.1699
[2700/6476] Loss: 5.7389
[2800/6476] Loss: 5.6990
[2900/6476] Loss: 5.3795
[3000/6476] Loss: 5.6113
[3100/6476] Loss: 5.2058
[3200/6476] Loss: 5.2377
[3300/6476] Loss: 5.5268
[3400/6476] Loss: 5.0578
[3500/6476] Loss: 5.7343
[3600/6476] Loss: 5.4385
[3700/6476] Loss: 5.5070
[3800/6476] Loss: 5.5028
[3900/6476] Loss: 5.2252
[4000/6476] Loss: 5.6118
[4100/6476] Loss: 5.7660
[4200/6476] Loss: 5.6998
[4300/6476] Loss: 4.8962
[4400/6476] Loss: 5.2822
[4500/6476] Loss: 5.2667
[4600/6476] Loss: 5.3542
[4700/6476] Loss: 5.0513
[4800/6476] Loss: 5.1663
[4900/6476] Loss: 5.3592
[5000/6476] Loss: 5.2747
[5100/6476] Loss: 5.4393
[5200/6476] Loss: 4.9865
[5300/6476] Loss: 5.8133
[5400/6476] Loss: 5.0365
[5500/6476] Loss: 5.0232
[5600/6476] Loss: 5.6425
[5700/6476] Loss: 5.7094
[5800/6476] Loss: 5.4407
[5900/6476] Loss: 5.5423
[6000/6476] Loss: 5.2228
[6100/6476] Loss: 5.4550
[6200/6476] Loss: 5.2122
[6300/6476] Loss: 5.5483
[6400/6476] Loss: 5.5610
	Loss:	35294.215
Recall@20 : 50.70955008547959 , MRR@20 17.37803426743242
-------------------------------------------------------
epoch:  2
[0/6476] Loss: 5.3373
[100/6476] Loss: 4.8832
[200/6476] Loss: 4.8623
[300/6476] Loss: 4.4894
[400/6476] Loss: 5.0387
[500/6476] Loss: 4.7041
[600/6476] Loss: 4.9128
[700/6476] Loss: 4.7225
[800/6476] Loss: 4.8435
[900/6476] Loss: 4.9317
[1000/6476] Loss: 5.0944
[1100/6476] Loss: 4.8418
[1200/6476] Loss: 4.9611
[1300/6476] Loss: 5.1427
[1400/6476] Loss: 4.6360
[1500/6476] Loss: 4.7179
[1600/6476] Loss: 5.0598
[1700/6476] Loss: 5.4515
[1800/6476] Loss: 5.2052
[1900/6476] Loss: 4.9529
[2000/6476] Loss: 4.6128
[2100/6476] Loss: 4.3721
[2200/6476] Loss: 4.3816
[2300/6476] Loss: 4.5994
[2400/6476] Loss: 5.3950
[2500/6476] Loss: 5.7185
[2600/6476] Loss: 4.4668
[2700/6476] Loss: 5.1265
[2800/6476] Loss: 5.3080
[2900/6476] Loss: 4.9954
[3000/6476] Loss: 5.0196
[3100/6476] Loss: 4.8500
[3200/6476] Loss: 4.7924
[3300/6476] Loss: 4.9899
[3400/6476] Loss: 4.7090
[3500/6476] Loss: 5.1282
[3600/6476] Loss: 4.9037
[3700/6476] Loss: 5.1751
[3800/6476] Loss: 5.0394
[3900/6476] Loss: 4.6484
[4000/6476] Loss: 5.1807
[4100/6476] Loss: 5.2835
[4200/6476] Loss: 5.3113
[4300/6476] Loss: 4.4994
[4400/6476] Loss: 4.8559
[4500/6476] Loss: 4.9343
[4600/6476] Loss: 4.9666
[4700/6476] Loss: 4.6028
[4800/6476] Loss: 4.8604
[4900/6476] Loss: 4.8287
[5000/6476] Loss: 4.9215
[5100/6476] Loss: 5.1380
[5200/6476] Loss: 4.6900
[5300/6476] Loss: 5.3448
[5400/6476] Loss: 4.6476
[5500/6476] Loss: 4.4925
[5600/6476] Loss: 5.2773
[5700/6476] Loss: 5.1834
[5800/6476] Loss: 4.9664
[5900/6476] Loss: 5.2171
[6000/6476] Loss: 4.8131
[6100/6476] Loss: 5.1338
[6200/6476] Loss: 4.9946
[6300/6476] Loss: 5.1906
[6400/6476] Loss: 5.2456
	Loss:	31812.346
Recall@20 : 51.06953729828901 , MRR@20 17.650087217232198
-------------------------------------------------------
epoch:  3
[0/6476] Loss: 5.1435
[100/6476] Loss: 4.5375
[200/6476] Loss: 4.5261
[300/6476] Loss: 4.1045
[400/6476] Loss: 4.5510
[500/6476] Loss: 4.2682
[600/6476] Loss: 4.4366
[700/6476] Loss: 4.1700
[800/6476] Loss: 4.4524
[900/6476] Loss: 4.5314
[1000/6476] Loss: 4.7758
[1100/6476] Loss: 4.3057
[1200/6476] Loss: 4.6051
[1300/6476] Loss: 4.7335
[1400/6476] Loss: 4.1722
[1500/6476] Loss: 4.3727
[1600/6476] Loss: 4.6848
[1700/6476] Loss: 4.8322
[1800/6476] Loss: 4.7930
[1900/6476] Loss: 4.4992
[2000/6476] Loss: 4.1728
[2100/6476] Loss: 4.0236
[2200/6476] Loss: 3.9765
[2300/6476] Loss: 4.1537
[2400/6476] Loss: 4.9487
[2500/6476] Loss: 5.2847
[2600/6476] Loss: 4.0067
[2700/6476] Loss: 4.6376
[2800/6476] Loss: 5.0064
[2900/6476] Loss: 4.7057
[3000/6476] Loss: 4.6295
[3100/6476] Loss: 4.5412
[3200/6476] Loss: 4.5463
[3300/6476] Loss: 4.5770
[3400/6476] Loss: 4.4125
[3500/6476] Loss: 4.6553
[3600/6476] Loss: 4.5051
[3700/6476] Loss: 4.8284
[3800/6476] Loss: 4.7211
[3900/6476] Loss: 4.3270
[4000/6476] Loss: 4.7647
[4100/6476] Loss: 4.9800
[4200/6476] Loss: 4.9502
[4300/6476] Loss: 4.0840
[4400/6476] Loss: 4.5321
[4500/6476] Loss: 4.5855
[4600/6476] Loss: 4.5647
[4700/6476] Loss: 4.2242
[4800/6476] Loss: 4.5474
[4900/6476] Loss: 4.4677
[5000/6476] Loss: 4.6776
[5100/6476] Loss: 4.8931
[5200/6476] Loss: 4.4139
[5300/6476] Loss: 4.9759
[5400/6476] Loss: 4.2420
[5500/6476] Loss: 4.1342
[5600/6476] Loss: 4.8965
[5700/6476] Loss: 4.7884
[5800/6476] Loss: 4.5877
[5900/6476] Loss: 4.8171
[6000/6476] Loss: 4.4347
[6100/6476] Loss: 4.7902
[6200/6476] Loss: 4.6956
[6300/6476] Loss: 4.8582
[6400/6476] Loss: 4.9081
	Loss:	29394.885
Recall@20 : 49.227903873684795 , MRR@20 17.043367669471017
-------------------------------------------------------
epoch:  4
[0/6476] Loss: 4.8954
[100/6476] Loss: 4.3076
[200/6476] Loss: 4.2554
[300/6476] Loss: 3.7963
[400/6476] Loss: 4.2262
[500/6476] Loss: 3.9364
[600/6476] Loss: 4.0813
[700/6476] Loss: 3.7034
[800/6476] Loss: 4.0531
[900/6476] Loss: 4.1981
[1000/6476] Loss: 4.3569
[1100/6476] Loss: 3.9111
[1200/6476] Loss: 4.3608
[1300/6476] Loss: 4.5181
[1400/6476] Loss: 3.9181
[1500/6476] Loss: 4.0370
[1600/6476] Loss: 4.2301
[1700/6476] Loss: 4.4351
[1800/6476] Loss: 4.3783
[1900/6476] Loss: 4.1638
[2000/6476] Loss: 3.8021
[2100/6476] Loss: 3.6732
[2200/6476] Loss: 3.6244
[2300/6476] Loss: 3.7934
[2400/6476] Loss: 4.5306
[2500/6476] Loss: 4.8319
[2600/6476] Loss: 3.7942
[2700/6476] Loss: 4.1713
[2800/6476] Loss: 4.6829
[2900/6476] Loss: 4.4386
[3000/6476] Loss: 4.3250
[3100/6476] Loss: 4.2064
[3200/6476] Loss: 4.3030
[3300/6476] Loss: 4.2356
[3400/6476] Loss: 4.1004
[3500/6476] Loss: 4.3024
[3600/6476] Loss: 4.2551
[3700/6476] Loss: 4.4466
[3800/6476] Loss: 4.4057
[3900/6476] Loss: 4.1021
[4000/6476] Loss: 4.4267
[4100/6476] Loss: 4.6256
[4200/6476] Loss: 4.5872
[4300/6476] Loss: 3.7423
[4400/6476] Loss: 4.2529
[4500/6476] Loss: 4.1557
[4600/6476] Loss: 4.3563
[4700/6476] Loss: 3.9673
[4800/6476] Loss: 4.2021
[4900/6476] Loss: 4.1226
[5000/6476] Loss: 4.4481
[5100/6476] Loss: 4.6435
[5200/6476] Loss: 4.1641
[5300/6476] Loss: 4.6361
[5400/6476] Loss: 3.9348
[5500/6476] Loss: 3.8359
[5600/6476] Loss: 4.4960
[5700/6476] Loss: 4.3850
[5800/6476] Loss: 4.2706
[5900/6476] Loss: 4.3891
[6000/6476] Loss: 4.0924
[6100/6476] Loss: 4.4552
[6200/6476] Loss: 4.3815
[6300/6476] Loss: 4.4699
[6400/6476] Loss: 4.4364
	Loss:	27272.623
Recall@20 : 46.77192933687297 , MRR@20 16.12103231500934
-------------------------------------------------------
epoch:  5
[0/6476] Loss: 4.5178
[100/6476] Loss: 4.0992
[200/6476] Loss: 3.9292
[300/6476] Loss: 3.5290
[400/6476] Loss: 4.0030
[500/6476] Loss: 3.6415
[600/6476] Loss: 3.7974
[700/6476] Loss: 3.2045
[800/6476] Loss: 3.7292
[900/6476] Loss: 3.9070
[1000/6476] Loss: 3.9984
[1100/6476] Loss: 3.6026
[1200/6476] Loss: 4.2039
[1300/6476] Loss: 4.2057
[1400/6476] Loss: 3.6513
[1500/6476] Loss: 3.7633
[1600/6476] Loss: 3.9309
[1700/6476] Loss: 4.2520
[1800/6476] Loss: 4.0472
[1900/6476] Loss: 3.8431
[2000/6476] Loss: 3.5049
[2100/6476] Loss: 3.3621
[2200/6476] Loss: 3.4133
[2300/6476] Loss: 3.5687
[2400/6476] Loss: 4.1366
[2500/6476] Loss: 4.4734
[2600/6476] Loss: 3.5603
[2700/6476] Loss: 3.7079
[2800/6476] Loss: 4.3278
[2900/6476] Loss: 4.2096
[3000/6476] Loss: 4.1117
[3100/6476] Loss: 3.8963
[3200/6476] Loss: 4.0080
[3300/6476] Loss: 3.9617
[3400/6476] Loss: 3.8094
[3500/6476] Loss: 3.9837
[3600/6476] Loss: 4.0588
[3700/6476] Loss: 4.1053
[3800/6476] Loss: 4.0606
[3900/6476] Loss: 3.8264
[4000/6476] Loss: 4.0926
[4100/6476] Loss: 4.3716
[4200/6476] Loss: 4.2979
[4300/6476] Loss: 3.4377
[4400/6476] Loss: 4.0822
[4500/6476] Loss: 3.9165
[4600/6476] Loss: 4.1462
[4700/6476] Loss: 3.7725
[4800/6476] Loss: 3.8757
[4900/6476] Loss: 3.8298
[5000/6476] Loss: 4.1670
[5100/6476] Loss: 4.4323
[5200/6476] Loss: 3.9500
[5300/6476] Loss: 4.3222
[5400/6476] Loss: 3.6630
[5500/6476] Loss: 3.5646
[5600/6476] Loss: 4.1120
[5700/6476] Loss: 4.0783
[5800/6476] Loss: 3.9936
[5900/6476] Loss: 4.0672
[6000/6476] Loss: 3.7814
[6100/6476] Loss: 4.1627
[6200/6476] Loss: 4.1192
[6300/6476] Loss: 4.1147
[6400/6476] Loss: 4.0831
	Loss:	25432.379
Recall@20 : 45.00535116127149 , MRR@20 15.505800721673799
-------------------------------------------------------
epoch:  6
[0/6476] Loss: 4.1555
[100/6476] Loss: 3.8894
[200/6476] Loss: 3.6339
[300/6476] Loss: 3.2964
[400/6476] Loss: 3.7019
[500/6476] Loss: 3.3947
[600/6476] Loss: 3.5062
[700/6476] Loss: 3.0504
[800/6476] Loss: 3.3869
[900/6476] Loss: 3.6543
[1000/6476] Loss: 3.7628
[1100/6476] Loss: 3.4464
[1200/6476] Loss: 4.0589
[1300/6476] Loss: 4.0419
[1400/6476] Loss: 3.4728
[1500/6476] Loss: 3.5147
[1600/6476] Loss: 3.7528
[1700/6476] Loss: 4.1302
[1800/6476] Loss: 3.9194
[1900/6476] Loss: 3.5671
[2000/6476] Loss: 3.3028
[2100/6476] Loss: 3.1982
[2200/6476] Loss: 3.2771
[2300/6476] Loss: 3.3134
[2400/6476] Loss: 3.7399
[2500/6476] Loss: 4.1659
[2600/6476] Loss: 3.4160
[2700/6476] Loss: 3.5031
[2800/6476] Loss: 4.0832
[2900/6476] Loss: 3.9851
[3000/6476] Loss: 3.9398
[3100/6476] Loss: 3.7022
[3200/6476] Loss: 3.7495
[3300/6476] Loss: 3.6657
[3400/6476] Loss: 3.5731
[3500/6476] Loss: 3.6827
[3600/6476] Loss: 3.8474
[3700/6476] Loss: 3.8546
[3800/6476] Loss: 3.8085
[3900/6476] Loss: 3.5809
[4000/6476] Loss: 3.8438
[4100/6476] Loss: 4.0719
[4200/6476] Loss: 4.0786
[4300/6476] Loss: 3.1022
[4400/6476] Loss: 3.8724
[4500/6476] Loss: 3.7978
[4600/6476] Loss: 3.9052
[4700/6476] Loss: 3.6339
[4800/6476] Loss: 3.7079
[4900/6476] Loss: 3.5356
[5000/6476] Loss: 3.9334
[5100/6476] Loss: 4.1817
[5200/6476] Loss: 3.7567
[5300/6476] Loss: 3.9764
[5400/6476] Loss: 3.4649
[5500/6476] Loss: 3.2173
[5600/6476] Loss: 3.8285
[5700/6476] Loss: 3.7364
[5800/6476] Loss: 3.7824
[5900/6476] Loss: 3.7234
[6000/6476] Loss: 3.5848
[6100/6476] Loss: 3.8191
[6200/6476] Loss: 3.8371
[6300/6476] Loss: 3.8245
[6400/6476] Loss: 3.6701
	Loss:	23892.520
Recall@20 : 43.56123257397807 , MRR@20 15.097915665563047
```

In [ ]:
    hit, mrr = [], []
    slices = test_data1.generate_batch(model.bmatch_size)
    for i in slices:
        alias_inputs, A, items, mask, targets = test_data1.get_slice(i)
        
        alias_inputs = tf.convert_to_tensor(alias_inputs)
        items = tf.convert_to_tensor(items)
        
        A = tf.convert_to_tensor(A)
        mask = tf.convert_to_tensor(mask)
        targets = tf.convert_to_tensor(targets)

        scores = model(items, A, mask,alias_inputs,training =False)
        scores = tf.math.top_k(scores,k=20).indices
        scores = scores.numpy()
        
            # sub_scores를 tensor에서 numpy로 바꾸고 아래는 그대로 실행하면 됨.
        for score, target, mask in zip(scores, targets, test_data1.mask):
            hit.append(np.isin(target - 1, score)) # 현재 target은 s_{n+1}인 item이므로, 
                # score 즉, 위에서 추출한 index가 일치하면 hit이라는 list에 append 해준다.

                # score vector에서 target-1과 일치하는 경우를 찾고, 없는 경우 0을 mrr이라는 list에 기록하고,
            if len(np.where(score == target - 1)[0]) == 0:
                mrr.append(0)
            else:
                # 있는 경우에는 (1/그 위치의 index +1)을 기록한다.
                mrr.append(1 / (np.where(score == target - 1)[0][0] + 1))
        print(i[0])
    hit = np.mean(hit) * 100
    mrr = np.mean(mrr) * 100

    print(f"Recall@20 : {hit} , MRR@20 {mrr}")

# tagnn, niser, srgnn, tagnn++ 다 거의 동일한 코드에서 build


이후 validation step에서 scores를 이용하여 topk를 구한 다음 절차를 진행한다.

```python
    for i in slices:
        targets, scores = forward(model, i, test_data)
        sub_scores = scores.topk(20)[1]
        sub_scores = trans_to_cpu(sub_scores).detach().numpy()
        for score, target, mask in zip(sub_scores, targets, test_data.mask):
            hit.append(np.isin(target - 1, score))
            if len(np.where(score == target - 1)[0]) == 0:
                mrr.append(0)
            else:
                mrr.append(1 / (np.where(score == target - 1)[0][0] + 1))
    hit = np.mean(hit) * 100
    mrr = np.mean(mrr) * 100
    return hit, mrr
```

- 

# tensorflow에도 top_k 있음.

[top_k](https://www.tensorflow.org/api_docs/python/tf/math/top_k)